-- create or replace TABLE prod.silver.JIRA_COMPONENTS_MAPPED as
-- select * from METRICS.STAGED.JIRA_COMPONENTS_MAPPED

In [ ]:
ALTER SESSION SET TIMEZONE = 'Europe/London';

In [ ]:

-- insert into prod.silver.jira_issues
-- select * from metrics.staged.jira_issues

In [ ]:
-- alter table prod.silver.jira_issues add GOALS varchar

In [ ]:
-- jira issues
insert into prod.silver.jira_issues
-- create or replace table prod.silver.jira_issues as
select
"id"::text AS ID,
"key"::text as key,
to_timestamp_tz(
            REGEXP_REPLACE(parse_json("versionedRepresentations"):"created"."1", '(\\d{2})(\\d{2})$', '\\1:\\2')
            ) AS CREATED,
to_timestamp_tz(
            REGEXP_REPLACE(parse_json("versionedRepresentations"):"updated"."1", '(\\d{2})(\\d{2})$', '\\1:\\2')
            ) AS UPDATED,
-- -- jira.value:"renderedFields"."created" as created,
-- -- jira.value:"renderedFields"."updated" as updated,
parse_json("versionedRepresentations"):"project"."1"."id"::TEXT as projectid, 

parse_json("versionedRepresentations"):"project"."1"."key"::TEXT as projectkey,
to_date(
            REGEXP_REPLACE(parse_json("versionedRepresentations"):"duedate"."1", '(\\d{2})(\\d{2})$', '\\1:\\2')
            ) AS due_date,
parse_json("versionedRepresentations"):"status"."1"."name"::text AS CURRENTSTATUS,  parse_json("versionedRepresentations"):"assignee"."1"."displayName"::text as ASSIGNEE,    
parse_json("versionedRepresentations"):"reporter"."1"."displayName"::text as reporter, 
to_timestamp_tz(
            REGEXP_REPLACE(parse_json("versionedRepresentations"):"resolutiondate"."1", '(\\d{2})(\\d{2})$', '\\1:\\2')
            ) AS resolution_date, 

parse_json("versionedRepresentations"):"resolutiondate"."1"::text as resolution_date_string, 
parse_json("versionedRepresentations"):"summary"."1"::text as summary, 
parse_json("versionedRepresentations"):"customfield_10006"."1"::int as storypoint, 
parse_json("versionedRepresentations"):"customfield_11328"."1"."value"::TEXT as OWNER_TEAM,
parse_json("versionedRepresentations"):"priority"."1"."name"::text as priority, 
parse_json("versionedRepresentations"):"customfield_11418"."1"."value"::TEXT as ISSUE_CATEGORY,

parse_json("versionedRepresentations"):"customfield_10017"."1"."value"::TEXT as impact,
parse_json("versionedRepresentations"):"customfield_11365"."1"."value"::TEXT as  ROOT_CAUSE_CATEGORY,
parse_json("versionedRepresentations"):"customfield_10206"."1"."value"::TEXT as CLIENT_NAME,
parse_json("fields"):"customfield_10028"::TEXT as ROOT_CAUSE,
parse_json("versionedRepresentations"):"components"."1"[0]."name"::text as COMPONENTS,
parse_json("fields"):"parent"."fields"."issuetype"."name"::TEXT as ISSUETYPE,


parse_json("versionedRepresentations"):"customfield_10004"."1"[0]."name"::TEXT AS SPRINTNAME,
parse_json("versionedRepresentations"):"customfield_10004"."1"[0]."id"::TEXT AS SPRINTID,
parse_json("versionedRepresentations"):"customfield_10004"."1"[0]."boardId"::TEXT AS BOARDID,
-- parse_json("versionedRepresentations"):"customfield_10004"."1"[0]."completeDate"::TEXT AS completeDate,

-- parse_json("versionedRepresentations"):"customfield_10004"."1"[0]."endDate"::TEXT AS endDate,
-- parse_json("versionedRepresentations"):"customfield_10004"."1"[0]."startDate"::TEXT AS startDate,
-- parse_json("versionedRepresentations"):"customfield_10004"."1"[0]."state"::TEXT AS state,
parse_json("versionedRepresentations"):"customfield_11443"."1"."value"::text as  Bug_Discovery_11443,
parse_json("versionedRepresentations"):"customfield_11898"."1"."value"::text as  Failure_Rate_11898,

parse_json("versionedRepresentations"):"customfield_11763"."1"."value"::variant as  Change_Failure_11763,
parse_json("versionedRepresentations"):"customfield_11310"."1"::text as  INHERENT_RISK_11310,

parse_json("versionedRepresentations"):"customfield_11575"."1"."value"::text as  INHERENT_RISK_LEVEL_11575,
-- parse_json("versionedRepresentations")::variant as vRepr,
current_timestamp as timestamp,
parse_json("versionedRepresentations"):"customfield_11765"."1"[0]."id"::text as  GOALS,

from prod.raw.jira_issues
-- WHERE GOALS IS NOT NULL
-- LATERAL FLATTEN(input => response_json) AS jira



-- insert into prod.silver.jira_issues
-- select * from metrics.staged.jira_issues

In [ ]:
INSERT OVERWRITE INTO prod.silver.jira_issues (
  ID,
  KEY,
  CREATED,
  UPDATED,
  PROJECTID,
  PROJECTKEY,
  DUE_DATE,
  CURRENTSTATUS,
  ASSIGNEE,
  REPORTER,
  RESOLUTION_DATE,
  RESOLUTION_DATE_STRING,
  SUMMARY,
  STORYPOINT,
    OWNER_TEAM,
  PRIORITY,

  ISSUE_CATEGORY,
  IMPACT,
  ROOT_CAUSE_CATEGORY,
  CLIENT_NAME,
  ROOT_CAUSE,

  COMPONENTS,
    ISSUETYPE,
  SPRINTNAME,
  SPRINTID,
  BOARDID,
  "Bug_Discovery_11443",
  "Failure_Rate_11898",
  "Change_Failure_11763",
  INHERENT_RISK_11310,
  INHERENT_RISK_LEVEL_11575,
  TIMESTAMP,
  GOALS
)
WITH ranked_issues AS (
  SELECT 
    ID,
    KEY,
    CREATED,
    UPDATED,
    PROJECTID,
    PROJECTKEY,
    DUE_DATE,
    CURRENTSTATUS,
    ASSIGNEE,
    REPORTER,
    RESOLUTION_DATE,
    RESOLUTION_DATE_STRING,
    SUMMARY,
    STORYPOINT,
      OWNER_TEAM,
    PRIORITY,
    ISSUE_CATEGORY,
    IMPACT,
    ROOT_CAUSE_CATEGORY,
    CLIENT_NAME,
    ROOT_CAUSE,

    COMPONENTS,
        ISSUETYPE,
    SPRINTNAME,
    SPRINTID,
    BOARDID,
  "Bug_Discovery_11443",
  "Failure_Rate_11898",
  "Change_Failure_11763",
    INHERENT_RISK_11310,
    INHERENT_RISK_LEVEL_11575,
    TIMESTAMP,
    GOALS,
    ROW_NUMBER() OVER (PARTITION BY ID ORDER BY UPDATED DESC) AS RN
  FROM prod.silver.jira_issues
)
SELECT 
  ID,
  KEY,
  CREATED,
  UPDATED,
  PROJECTID,
  PROJECTKEY,
  DUE_DATE,
  CURRENTSTATUS,
  ASSIGNEE,
  REPORTER,
  RESOLUTION_DATE,
  RESOLUTION_DATE_STRING,
  SUMMARY,
  STORYPOINT,
    OWNER_TEAM,
  PRIORITY,
  ISSUE_CATEGORY,
  IMPACT,
  ROOT_CAUSE_CATEGORY,
  CLIENT_NAME,
  ROOT_CAUSE,

  COMPONENTS,
    ISSUETYPE,
  SPRINTNAME,
  SPRINTID,
  BOARDID,
  "Bug_Discovery_11443",
  "Failure_Rate_11898",
  "Change_Failure_11763",
  INHERENT_RISK_11310,
  INHERENT_RISK_LEVEL_11575,
  TIMESTAMP,
  GOALS
FROM ranked_issues
WHERE RN = 1;


In [ ]:
-- -- -- -- jira issues changes
insert into prod.silver.jira_issues_changes 
SELECT 
"id"::int as issueid,
"key"::text as KEY,
DATE(SUBSTRING(parse_json("changelog"):"histories"[0]."created"::string, 1, CHARINDEX('T', parse_json("changelog"):"histories"[0]."created"::string) - 1)) AS createddate,
parse_json("changelog"):"histories"[0]."id"::string as changeid,
parse_json("changelog"):"histories"[0]."items"[0]."fromString"::string  AS fromState,
parse_json("changelog"):"histories"[0]."items"[0]."toString"::string AS toState,
current_timestamp as timestamp
FROM
prod.raw.jira_issues
WHERE parse_json("changelog"):"histories"[0]."items"[0]."field"::string = 'status';
-- where jira.value:"key" = 'RM-1652'

In [ ]:
INSERT OVERWRITE INTO prod.silver.jira_issues_changes
WITH ranked_issues AS (
  SELECT 
    issueid,
    KEY,
    createddate,
    changeid,
    fromState,
    toState,
    timestamp,
    ROW_NUMBER() OVER (
      PARTITION BY (issueid || '-' || KEY || '-' || createddate || '-' || changeid || '-' || fromState || '-' || toState)
      ORDER BY timestamp DESC
    ) AS RN
  FROM prod.silver.jira_issues_changes
)
SELECT 
  issueid,
  KEY,
  createddate,
  changeid,
  fromState,
  toState,
  timestamp
FROM ranked_issues
WHERE RN = 1;


In [ ]:
insert into prod.silver.JIRA_ISSUES_CHANGES_OwnerTeam_TLS 
SELECT 
"id"::int as ISSUEID,
"key"::text as KEY,
DATE(SUBSTRING(parse_json("changelog"):"histories"[0]."created"::string, 1, CHARINDEX('T', parse_json("changelog"):"histories"[0]."created"::string) - 1)) AS CREATEDDATE,
parse_json("changelog"):"histories"[0]."id"::string as changeid,
parse_json("changelog"):"histories"[0]."items"[0]."fromString"::string  AS FROMOWNERTEAM,
parse_json("changelog"):"histories"[0]."items"[0]."toString"::string AS TOOWNERTEAM,
current_timestamp as timestamp
FROM
prod.raw.jira_issues
WHERE 
-- parse_json("changelog"):"histories"[0]."items"[0]."field"::string = 'Owner Team' and 
"key" like 'TLS%';

In [ ]:
INSERT OVERWRITE INTO prod.silver.JIRA_ISSUES_CHANGES_OwnerTeam_TLS
WITH ranked_issues AS (
  SELECT 
    issueid,
    KEY,
    CREATEDDATE,
    changeid,
    FROMOWNERTEAM,
    TOOWNERTEAM,
    timestamp,
    ROW_NUMBER() OVER (
      PARTITION BY (issueid || '-' || KEY || '-' || CREATEDDATE || '-' || changeid || '-' || FROMOWNERTEAM || '-' || TOOWNERTEAM)
      ORDER BY timestamp DESC
    ) AS RN
  FROM prod.silver.JIRA_ISSUES_CHANGES_OwnerTeam_TLS
)
SELECT 
    issueid,
    KEY,
    CREATEDDATE,
    changeid,
    FROMOWNERTEAM,
    TOOWNERTEAM,
    timestamp,
FROM ranked_issues
WHERE RN = 1;

In [ ]:
-- jira projects

    CREATE OR REPLACE TABLE prod.silver.JIRA_PROJECTS as
SELECT
id,
key,
name,
isprivate,
projecttypekey
,current_timestamp as timestamp
FROM prod.raw.JIRA_PROJECTS;

select * from prod.silver.JIRA_PROJECTS;

In [ ]:
-- jira boards
-- CREATE OR REPLACE table prod.silver.jira_boards
-- (
-- id int,
-- name varchar,
-- type varchar,
-- projectid int, 
-- projectkey varchar,
-- sprintid int,
-- currentsprinturl varchar,
-- timestamp timestamp_tz
-- );

truncate prod.silver.jira_boards;

insert into prod.silver.jira_boards
WITH sprintBoard AS (
 SELECT
    sprintId,
    boardid
  FROM
    metrics.staged.jira_issues 
  group by 
  sprintId,boardid
  having boardid is not NULL
)
SELECT 
Id,
Name,
Type,
ProjectId,
ProjectKey,
sb.sprintid,
CONCAT('https://phlexglobal.atlassian.net/jira/software/c/projects/', ProjectKey , '/boards/' , Id,'?sprints=',sb.sprintid) AS CurrentSprintUrl
  ,current_timestamp as timestamp
FROM prod.raw.jira_boards b
join sprintBoard sb ON sb.boardid = b.Id;

-- select * from prod.silver.jira_boards

In [ ]:
-- jira sprints

-- # ----- # ----- # -----# ----- #

-- CREATE OR REPLACE table prod.silver.jira_sprints
-- (id int,
-- name varchar,
-- state varchar,
-- enddate date,
-- startdate date,
-- createddate date,
-- completedate varchar,
-- timestamp timestamp_tz);

truncate table prod.silver.jira_sprints;

insert into prod.silver.jira_sprints
SELECT distinct
id,
name,
state,
enddate,
startdate,
createddate,
completedate
  ,current_timestamp as timestamp
FROM prod.raw.jira_sprints;

select * from prod.silver.jira_sprints
where state <> 'closed'

In [ ]:
-- jira project versions

-- CREATE OR REPLACE TABLE prod.silver.JIRA_PROJECT_VERSIONS (
--     id STRING,
--     name STRING,
--     self STRING,
--     overdue BOOLEAN,
--     archived BOOLEAN,
--     released BOOLEAN,
--     projectid STRING,
--     startdate DATE,
--     description STRING,
--     releasedate DATE,
--     userstartdate DATE,
-- timestamp timestamp_tz);


truncate table prod.silver.JIRA_PROJECT_VERSIONS;

insert into prod.silver.JIRA_PROJECT_VERSIONS
SELECT
id,
name,
self,
overdue,
archived,
released,
projectid,
startdate,
description,
releasedate,
userstartdate
  ,current_timestamp as timestamp
FROM prod.raw.JIRA_PROJECT_VERSIONS;

select * from prod.silver.JIRA_PROJECT_VERSIONS;

In [ ]:
-- -- jira change lead time

CREATE OR REPLACE TABLE prod.silver.JIRA_CHANGE_LEAD_TIME
AS
SELECT 
ic.issueid AS issueId,
MAX(CASE WHEN fromstate = 'To Do' AND tostate <> 'To Do' THEN createddate END) AS toDoDate,
MAX(CASE WHEN fromstate <> 'Done' AND tostate = 'Done' THEN createddate END) AS doneDate,
CAST(TIMESTAMPDIFF(DAY, 
        MAX(CASE WHEN fromstate = 'To Do' AND tostate <> 'To Do' THEN createddate END),
        MAX(CASE WHEN fromstate <> 'Done' AND tostate = 'Done' THEN createddate END)
    )AS INTEGER)+1 AS duration
      ,current_timestamp as timestamp
FROM prod.silver.JIRA_ISSUES_CHANGES ic
JOIN prod.silver.JIRA_ISSUES i on ic.issueid=i.id
WHERE i.currentstatus='Done'
AND (fromstate = 'To Do' OR tostate = 'Done')
GROUP BY
ic.issueid,i.currentstatus
HAVING
ToDoDate IS NOT NULL AND DoneDate IS NOT NULL;

select * from prod.silver.JIRA_CHANGE_LEAD_TIME

In [ ]:
CREATE OR REPLACE TABLE prod.silver.JIRA_SPRINT_ISSUES as
SELECT
id,
ISSUE_KEY as key,
ISSUE_ID as issueid,
sprint_id as sprintid,
created,
updated
  ,current_timestamp as timestamp
FROM prod.raw.JIRA_SPRINT_ISSUES_SNOWFLAKE;

select * from prod.silver.JIRA_SPRINT_ISSUES;

In [ ]:
-- jira sprint velocity

-- CREATE OR REPLACE TABLE prod.silver.JIRA_SPRINT_VELOCITY (
--     sprintid STRING,
--    sprintname STRING,
--     startdate DATE,
--     enddate DATE,
--     sprint_duration INT,
--     issues_created INT,
--     issues_completed INT,
--     velocity FLOAT,
--     sprint_success_rate_perc FLOAT
-- );


truncate table prod.silver.JIRA_SPRINT_VELOCITY;

insert into prod.silver.JIRA_SPRINT_VELOCITY
WITH completed_issues AS (
 SELECT
    s.id as sprintid,
    s.name as sprintname,
    COUNT(DISTINCT i.key) as issues_completed
  FROM
        prod.silver.JIRA_SPRINTS s
  JOIN
    prod.silver.JIRA_SPRINT_ISSUES i ON s.id=i.sprintid
JOIN prod.silver.jira_issues_changes ic on ic.createddate BETWEEN DATE(s.startdate) AND DATE(s.completedate)
AND ic.issueid=i.issueid
    AND ic.tostate = 'Done'
    GROUP BY
    s.id,
    s.name
)

SELECT
  s.id as sprintid,
  s.name as sprintname,
  DATE(s.startdate) as startdate,
  DATE(s.completedate) enddate,
  DATEDIFF(day, s.startdate, s.completedate) AS sprint_duration,
  COUNT(DISTINCT i.issueid) AS issues_created,
  COALESCE(ci.issues_completed, 0) AS issues_completed,
  COALESCE(ci.issues_completed, 0)/DATEDIFF(day, s.startdate, s.completedate) as velocity,
  COALESCE(ci.issues_completed / NULLIF(issues_created, 0), 0)*100 AS sprint_success_rate_perc
FROM
  prod.silver.JIRA_SPRINTS s
  JOIN
    prod.silver.JIRA_SPRINT_ISSUES i ON s.id = i.sprintid
LEFT JOIN
  completed_issues ci ON s.id = ci.sprintid
  WHERE s.startdate >= '2024-01-01'
  AND s.state = 'closed'
  AND DATEDIFF(day, s.startdate, s.completedate) > 0
  GROUP BY 
  s.id,
  s.name,
  s.startdate,
  s.completedate,
  ci.issues_completed;


In [ ]:
select * from prod.silver.JIRA_SPRINT_VELOCITY

In [ ]:
-- jira sprint velocity open

-- CREATE OR REPLACE TABLE prod.silver.JIRA_SPRINT_VELOCITY_OPEN (
--     sprintid STRING,
--     sprintname STRING,
--     storypoint_completed FLOAT,
--     all_completed FLOAT,
--     startdate DATE,
--     enddate DATE,
--     sprint_duration INT,
--     daysremaining INT,
--     issues_created INT,
--     issues_completed INT,
--     velocity FLOAT,
--     sprint_success_rate_perc FLOAT,
-- timestamp timestamp_tz);

    truncate table prod.silver.JIRA_SPRINT_VELOCITY_OPEN;

insert into prod.silver.JIRA_SPRINT_VELOCITY_OPEN
        WITH completed_issues AS (
        
        Select sprintid,sprintname,sum(issues_completed) as issues_completed,sum(storypoint) as storypoint_completed from
        (
         SELECT
            s.id as sprintid,
            s.name as sprintname,
            COUNT(DISTINCT i.key) as issues_completed,
            i.key,
            --ji.boardid as boardid,
            (Select sum(storypoint) as storypoint  from (select distinct ji.id,ji.key, ji.storypoint as storypoint  from prod.silver.jira_issues ji where ji.currentstatus = 'Done') where key =i.key ) as storypoint
          FROM
            prod.silver.JIRA_SPRINTS s
          Left JOIN prod.silver.JIRA_SPRINT_ISSUES i ON s.id=i.sprintid
          JOIN prod.silver.jira_issues ji ON ji.key =i.key and ji.currentstatus = 'Done'
          GROUP BY
            s.id,
            s.name,
            i.key
            --ji.boardid
        ) group by sprintid,sprintname,issues_completed--,boardid
        
        ),
        all_issues AS (
        Select sprintid,sum(storypoint) as storypoint_completed from
        
            prod.silver.jira_issues
          GROUP BY
            sprintid
        )
        SELECT
          s.id as sprintid,
          s.name as sprintname,
          COALESCE(ci.storypoint_completed, 0)  as storypoint_completed,
          COALESCE(ai.storypoint_completed, 0)  as all_completed,
          DATE(s.startdate) as startdate,
          DATE(s.enddate) enddate,
          DATEDIFF(day, s.startdate, s.enddate) AS sprint_duration,
          DATEDIFF(day,  Getdate(),s.enddate) AS daysremaining,
          COUNT(DISTINCT i.issueid) AS issues_created,
          COALESCE(ci.issues_completed, 0) AS issues_completed,
          COALESCE(ci.issues_completed, 0)/DATEDIFF(day, s.startdate, Getdate()) as velocity,
          COALESCE(ci.issues_completed / NULLIF(issues_created, 0), 0)*100 AS sprint_success_rate_perc
            ,current_timestamp as timestamp
        FROM prod.silver.JIRA_SPRINTS s
          JOIN prod.silver.JIRA_SPRINT_ISSUES i ON s.id = i.sprintid
          JOIN prod.silver.jira_issues ji ON ji.id =i.issueid
          LEFT JOIN completed_issues ci ON s.id = ci.sprintid
          LEFT JOIN all_issues ai ON s.id = ai.sprintid
          WHERE s.startdate >= '2024-01-01'
          AND s.state = 'active'
           AND 
          DATEDIFF(day, s.startdate, Getdate()) > 0
          GROUP BY 
          s.id,
          s.name,
          s.startdate,
          s.enddate,
          s.completedate,
          ci.issues_completed,
          ci.storypoint_completed,
          ai.storypoint_completed;

  select * from prod.silver.JIRA_SPRINT_VELOCITY_OPEN

In [ ]:
select * from prod.silver.JIRA_SPRINT_VELOCITY_OPEN

In [ ]:
-- RiskManager
insert into prod.silver.jira_risk_manager 
SELECT
parse_json("versionedRepresentations"):"customfield_11664"."1"."displayName"::string as USER_NAME,
"key"::text as ISSUE_KEY,
parse_json("versionedRepresentations"):"customfield_11664"."1"."accountId"::string as USER_ACCOUNT_ID
,current_timestamp as timestamp
FROM prod.raw.JIRA_ISSUES
where USER_NAME is not null;

In [ ]:
INSERT OVERWRITE INTO prod.silver.jira_risk_manager 
WITH ranked_issues AS (
  SELECT 
USER_NAME,
ISSUE_KEY,
USER_ACCOUNT_ID,
    TIMESTAMP,
    ROW_NUMBER() OVER (PARTITION BY (USER_NAME || '-' ||
ISSUE_KEY|| '-' ||
USER_ACCOUNT_ID) ORDER BY TIMESTAMP DESC) AS RN
  FROM prod.silver.jira_risk_manager 
)
SELECT 
USER_NAME,
ISSUE_KEY,
USER_ACCOUNT_ID,
    TIMESTAMP,
FROM ranked_issues
WHERE RN = 1;


In [ ]:
-- Targets table manual input:
CREATE OR REPLACE TABLE prod.silver.jira_targets (
    "Zero" INT, 
    "Hundred" INT, 
    "SLA_Target" FLOAT, 
    "Regression_Target" INT, 
    "MTTR_Target" INT, 
    "Steps_Target" FLOAT, 
    "RiskMTTR_Target" INT, 
    "NA" INT, 
    "DevAzure_Target" INT, 
    "ProdAzure_Target" INT, 
    "OtherAzure_Target" INT, 
    "PercSupported_Target" FLOAT, 
    "Velocity_Target" FLOAT, 
    "Int_Target" INT, 
    "AutoInt_Target" INT, 
    "Unit_Target" INT, 
    "AutoUnit_Target" INT
);

INSERT INTO prod.silver.jira_targets (
    "Zero", 
    "Hundred", 
    "SLA_Target", 
    "Regression_Target", 
    "MTTR_Target", 
    "Steps_Target", 
    "RiskMTTR_Target", 
    "NA", 
    "DevAzure_Target", 
    "ProdAzure_Target", 
    "OtherAzure_Target", 
    "PercSupported_Target", 
    "Velocity_Target", 
    "Int_Target", 
    "AutoInt_Target", 
    "Unit_Target", 
    "AutoUnit_Target"
) VALUES (
    0, 
    100,
    99.95, 
    5, 
    5, 
    1.5, 
    90, 
    -1, 
    67000, 
    125000, 
    7000, 
    95, 
    0.8, 
    0, 
    0, 
    0, 
    0
);


In [ ]:
-- create or replace table prod.silver.jira_risk_types as
-- select * from metrics.staged.di_kpi_risk_types;
-- select * from prod.silver.jira_risk_types


In [ ]:
-- RiskTypes
insert into prod.silver.jira_risk_types 
SELECT
"key"::text as ISSUE_KEY,
parse_json("versionedRepresentations"):"customfield_11570"."1"[0]."value"::string as "Risk_Type",

current_timestamp as timestamp
FROM prod.raw.JIRA_ISSUES
where "Risk_Type" is not null;

select * from prod.silver.jira_risk_types 

In [ ]:
INSERT OVERWRITE INTO prod.silver.jira_risk_types
WITH ranked_issues AS (
  SELECT 
ISSUE_KEY,
"Risk_Type",
    TIMESTAMP,
    ROW_NUMBER() OVER (PARTITION BY ("Risk_Type" || '-' ||
ISSUE_KEY) ORDER BY TIMESTAMP DESC) AS RN
  FROM prod.silver.jira_risk_types
)
SELECT 
ISSUE_KEY,
"Risk_Type",
    TIMESTAMP,
FROM ranked_issues
WHERE RN = 1;


In [ ]:
select * from PROD.SILVER.JIRA_ISSUES
WHERE GOALS is not null

In [ ]:
-- create table prod.silver.PRODUCTS as
-- select * from METRICS.STAGED.PRODUCTS

In [ ]:
-- Create or Replace TABLE prod.silver.PRODUCTS_SILVER AS SELECT * FROM METRICS.STAGED.PRODUCTS;